In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Get the current research's directory
research_dir = os.path.dirname(os.path.abspath('__file__'))

# Move one directory back
parent_dir = os.path.dirname(research_dir)

# Change the current working directory to the parent directory
os.chdir(parent_dir)

# Print the current working directory to confirm
print(f"Current working directory: {os.getcwd()}")

In [ ]:
from frank.utils.common import print_process_astream

# Base Agent in LangGraph

#### LLM Services

In [ ]:
from services.ai_foundry.llm import LLMServices

LLMServices.launch()

#### TOOLs

In [ ]:
import requests
import random
from typing import List
from langchain_core.tools import ToolException
from langchain_core.tools import tool

@tool
def get_evolution(pokemon_name: str) -> list:
    """This is a method to give you a information of the evolution path of a certain pokemon

    Args:
        pokemon_name: a pokemon name given by the user.
    """

    species_url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_name.lower()}"
    species_response = requests.get(species_url)

    if species_response.status_code != 200:
        raise ToolException(f"Error: {pokemon_name} is not a valid pokemon")
    
    species_data = species_response.json()

    # Step 2: Extract evolution chain URL from species data
    evolution_chain_url = species_data['evolution_chain']['url']

    # Step 3: Get the evolution chain data
    evolution_response = requests.get(evolution_chain_url)
    evolution_data = evolution_response.json()

    # Step 4: Traverse the evolution chain and get the names of evolutions
    evolutions = []
    current_evolution = evolution_data['chain']
    
    while current_evolution:
        evolutions.append(current_evolution['species']['name'])
        if len(current_evolution['evolves_to']) > 0:
            current_evolution = current_evolution['evolves_to'][0]
        else:
            break
    
    return evolutions

@tool
def random_movements(pokemon_name: str) -> List[str]:
    """This is a method to give you a random movements list of a certain pokemon if the user asks for them

    Args:
        pokemon_name: a pokemon name given by the user.
    """

    # The url of the api
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}'
        
    # Make the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        raise ToolException(f"Error: {pokemon_name} is not a valid pokemon")

    # Parse the response JSON
    data = response.json()

    # Extract the list of moves using map and lambda
    moves = list(map(lambda move: move['move']['name'], data['moves']))

    if len(moves) < 4:
        return moves

    # Select 4 random
    selected_moves = random.sample(moves, 4)

    return selected_moves

tools = [get_evolution, random_movements]

#### PROMPT & CHAIN

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder,
)

system_prompt="""
<context>
You are Professor Oak, a world-renowned Pokémon Professor from Pallet Town. Your expertise lies exclusively in Pokémon, 
and you have very limited knowledge of real-world animals.
</context>

<instructions> 
1. If you know the answer, respond confidently and clearly.
2. Keep your answers short and to the point and avoid referring back to earlier discussions.
3. Whenever someone mentions an animal, you will assume they are referring to a Pokémon that closely resembles that animal. 
4. You will describe the Pokémon in detail, including its type, abilities, habitat, and any unique traits it has, as if it is the animal in question. 
5. You should always try to connect it back to your vast knowledge of Pokémon.
</instructions>
    
<input>
strings
</input>
    
<output_format>
strings
</output_format>

<restrictions>
1. You dont know about real animals—only Pokémon.
</restrictions>
"""

history_template = [
    ("human", "Professor Oak, I want to be Pokémon Trainer and catch 'em all!"), 
    ("ai", """Ah, so you want to become a Pokémon Trainer, do you? That's quite the ambitious goal!
    First things first, you will need a partner. Have you thought about which starter Pokémon you want to choose?"""),
]


prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            # few_shot_prompt,
            history_template[0],
            MessagesPlaceholder(variable_name="messages"),
        ])


model_with_tools = LLMServices.model.bind_tools(tools)

chain = prompt_template | model_with_tools

In [ ]:
# Test the model without langgraph
# Define the input
user_input = "Hello, how are you?"
message_input = {"messages": [{"role": "human", "content": user_input}]}

chain.invoke(message_input)

In [ ]:
# Define the input
user_input = "What is the evolution of Pikachu?"
message_input = {"messages": [{"role": "human", "content": user_input}]}

chain.invoke(message_input)

#### GRAPH COMPONENTS

In [ ]:
from typing import Literal
from langgraph.graph import MessagesState

# NOTE: this is a method 'from langgraph.prebuilt import tools_condition'
def tools_condition(state, messages_key: str = "messages") -> Literal["end", "tools"]:
    if isinstance(state, list):
        ai_message = state[-1]
    elif isinstance(state, dict) and (messages := state.get(messages_key, [])):
        ai_message = messages[-1]
    elif messages := getattr(state, messages_key, []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "end"

def call_model_chained(state):
    messages = state["messages"]
    response = chain.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

#### GRAPH COMPILE

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import InMemorySaver

workflow = StateGraph(MessagesState)

# Define the function to execute tools
tool_node = ToolNode(tools)

# Define the checkpointer
memory = InMemorySaver()

# Nodes
workflow.add_node("OakLangAgent", call_model_chained)
workflow.add_node("OakTools", tool_node)

# Edges
workflow.add_edge(START, "OakLangAgent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "OakLangAgent",
    tools_condition,
    {
        "tools": "OakTools",
        "end": END,
    },
)

workflow.add_edge("OakTools", "OakLangAgent")

# Finally, we compile it!
graph = workflow.compile(checkpointer=memory)

In [ ]:
# Define the input
user_input = "Hi!"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config={"configurable": {"thread_id": "001"}})

In [ ]:
# Define the input
user_input = "Could you give me random movements of pickachu, and what is his evolutions?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config={"configurable": {"thread_id": "001"},})

In [ ]:
# Define the input
user_input = "Please, give me random movements of that evolution?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config={"configurable": {"thread_id": "001"},})

# Base Agent in LangGraph + Frank utilities

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from frank.workflow_builder import WorkflowBuilder
from frank.config.layouts.simple_oak_config_graph import SimpleOakConfigGraph
from frank.models.stategraph.stategraph import SharedState
from frank.utils.common import read_yaml
from frank.utils.logger import setup_logging
from frank.constants import *

## Read the config.yaml
config = read_yaml(CONFIG_FILE_PATH)

## Setup logging Configuration
setup_logging(config)

## Workflow Configuration for the main graph
workflow_builder = WorkflowBuilder(
    config=SimpleOakConfigGraph, 
    state_schema=SharedState,
    checkpointer=InMemorySaver()
)
graph = workflow_builder.compile() # compile the graph

In [ ]:
# Define the input
user_input = "Could you give me random movements of pickachu, and what is his evolutions?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config={"configurable": {"thread_id": "001"},})

In [ ]:
# Define the input
user_input = "Please, give me random movements of that evolution?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config={"configurable": {"thread_id": "001"},})

# LangGraph + Opentelemetry tracer

In [ ]:
### Azure AI Inference Tracer
import os
from langchain_azure_ai.callbacks.tracers import AzureAIInferenceTracer

## MANAGE IDENTITY INSTEAD ENV
# from azure.ai.projects import AIProjectClient
# from azure.identity import DefaultAzureCredential # NOTE USE ASYNC DEFAULT CREDENTIALS

# project_client = AIProjectClient.from_connection_string(
#     credential=DefaultAzureCredential(),
#     conn_str="<your-project-connection-string>",
# )

## Runnable_Config + Langchain Tracer
langchain_tracer = AzureAIInferenceTracer(
    connection_string=os.environ["AZURE_APP_INSIGHT_CONNECTION_STRING"], # project_client.telemetry.get_connection_string()
    enable_content_recording=True,
)

runnable_config = {
    "configurable": {"thread_id": "002"}, 
    "callbacks": [langchain_tracer]}

In [ ]:
# Define the input
user_input = "Could you give me some random moves for Pikachu, as well as some random moves for Primeape's evolution?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
await print_process_astream(graph, message_input, runnable_config=runnable_config)